In [ ]:
import os
import sys
import glob
import pickle
sys.path.append('../metanas')

import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('ggplot')

from metanas.utils.visualize import plot
from IPython.display import Image, display, Markdown

matplotlib.rcParams['font.serif'] = ['CMU Serif Roman']

In [ ]:
def plot_accuracy_loss_multiple_seeds(paths, labels, every=5, title=""):
    def get_results(path):
        result = []
        for p in glob.glob(f"{path}/*/experiment.pickle"):
            with (open(p, 'rb')) as f:
                res = pickle.load(f)
                result.append(res)
        return result

    def plot_line_training(ax, result, indice, label, y_label):
        train_acc = np.array([np.array(m[indice]) for m in result])

        len_epoch = len(np.array(result[0][indice]))
        train_acc_mean = train_acc.mean(axis=0)
        train_acc_std = train_acc.std(axis=0)
        
        x_axis_meta = np.array(range(10))
        
        ax.plot(x_axis_meta, train_acc_mean, lw = 2, alpha = 1, label=label)
        ax.set_title(f"Meta-training {y_label}")
        ax.fill_between(x_axis_meta, train_acc_mean - train_acc_std,
                        train_acc_mean + train_acc_std, alpha = 0.4)
        ax.set_xlabel("Epoch")
        ax.set_ylabel(y_label)
        
        handles, labels = ax.get_legend_handles_labels()
        ax.legend(handles, labels, loc="best")
    
    def plot_line_test(ax, result, indice, label, y_label):
        test_spacing = np.linspace(every, len(result[0][indice]),
                                   num=len(result[0][indice]), 
                                   retstep=every, dtype=np.int32)[0]

        train_acc = np.array([np.array(m[indice]) for m in result])

        len_epoch = len(np.array(result[0][indice]))
        train_acc_mean = train_acc.mean(axis=0)
        train_acc_std = train_acc.std(axis=0)
        
        ax.plot(test_spacing[:len(result[0][indice])],
                train_acc_mean,
                lw = 2,
                alpha = 1,
                label=label)
        ax.fill_between(test_spacing[:len(result[0][indice])],
                        train_acc_mean - train_acc_std,
                        train_acc_mean + train_acc_std, alpha = 0.4)
        ax.set_title(f"Meta-testing {y_label}")
        ax.set_xlabel("epoch")
        ax.set_ylabel(y_label)
        
        handles, labels = ax.get_legend_handles_labels()
        ax.legend(handles, labels, loc="best")
    
    results = [get_results(p) for p in paths]
    
    fig, ax = plt.subplots(2, 2, figsize=(20, 10))
    plt.subplots_adjust(hspace=0.3)# left=None, bottom=None, right=None, top=None, wspace=None, hspace=None)
    plt.suptitle(title, fontsize=14)
    
    y_label = ["acurracy", "loss"]
    for i, res in enumerate(results):
        for j, indice in enumerate(["train_test_accu", "train_test_loss"]):
            plot_line_training(ax[j, 0], res, indice, labels[i], y_label=y_label[j])
        
        for j, indice in enumerate(["test_test_accu", "test_test_loss"]):
            plot_line_test(ax[j, 1], res, indice, labels[i], y_label=y_label[j])


paths_sharp = [
    "/home/rob/Git/meta-fsl-nas/metanas/results/ablation/omniglot_train_metanas",
    "/home/rob/Git/meta-fsl-nas/metanas/results/ablation/omniglot_train_ss_sharp",
    "/home/rob/Git/meta-fsl-nas/metanas/results/ablation/omniglot_train_alpha_reg",
    "/home/rob/Git/meta-fsl-nas/metanas/results/ablation/omniglot_train_power_cosine",
]

names_sharp = [
    "MetaNAS Baseline",
    "SharpDARTS Search Space",
    "Alpha Regularization",
    "Cosine Power Annealing",
]

plot_accuracy_loss_multiple_seeds(paths_sharp, names_sharp,
                   title="Ablation sharpDARTS study, n=1, k=20, Omniglot dataset")

In [ ]:
paths = [
    "/home/rob/Git/meta-fsl-nas/metanas/results/ablation/omniglot_train_metanas",
    "/home/rob/Git/meta-fsl-nas/metanas/results/ablation/omniglot_train_ssa",
    "/home/rob/Git/meta-fsl-nas/metanas/results/ablation/omniglot_train_ssr",
]

names = [
    "MetaNAS Baseline",
    "Search Space Approximation",
    "Search Space Regularization"
]


plot_accuracy_loss_multiple_seeds(paths, names,
                   title="Ablation P-DARTS study, n=1, k=20, Omniglot dataset")

In [ ]:
# TODO: Compare baseline vs best ablation study additions

paths = [
    "/home/rob/Git/meta-fsl-nas/metanas/results/ablation/omniglot_train_metanas",
]

names = [
    "MetaNAS Baseline",
]

plot_accuracy_loss_multiple_seeds(paths, names,
                   title="Baseline comparison, n=1, k=20, Omniglot dataset")

In [ ]:
# TODO: Compare transfer of dataset statistics

## Plots single runs
Deprecated plots of visualizing single runs

In [ ]:
def plot_loss_accuracies(paths, names, eval_every=5, title=""):
    results = []
    palette = sns.color_palette()
    palette

    for p in paths:
        with (open(p, 'rb')) as f:
            res = pickle.load(f)
            results.append(res)
    
    _, axes = plt.subplots(2, 2, figsize=(20, 10))
    plt.suptitle(title, fontsize=14)
    test_spacing = np.linspace(eval_every, len(results[0]['train_test_loss']),
                               num=len(results[0]['test_test_loss']), 
                               retstep=eval_every, dtype=np.int32)[0]

    for i, res in enumerate(results):
        color = palette.pop()
        axes[0, 0].set_title("Meta-training loss")
        axes[0, 0].plot(res['train_test_loss'], 'o-',
                        color=color, label=f"{names[i]} loss")
        axes[0, 0].set_ylabel("Loss")
        axes[0, 0].set_xlabel("Epochs")
        axes[0, 0].legend(loc="best")
        
        axes[0, 1].set_title("Meta-test loss")
        axes[0, 1].plot(test_spacing[:len(res['test_test_loss'])],
                        res['test_test_loss'],  '^-', color=color,
                        label=f"{names[i]} loss")
        axes[0, 1].set_ylabel("Loss")
        axes[0, 1].set_xlabel("Epochs")
        axes[0, 1].legend(loc="best")

        axes[1, 0].set_title("Meta-training accuracy")
        axes[1, 0].plot(res['train_test_accu'], 'o-', color=color,
                        label=f"{names[i]} accuracy")
        axes[1, 0].set_ylabel("Accuracy")
        axes[1, 0].set_xlabel("Epochs")
        axes[1, 0].legend(loc="best")

        axes[1, 1].set_title("Meta-test accuracy")
        axes[1, 1].plot(test_spacing[:len(res['test_test_accu'])],
                        res['test_test_accu'], '^-', color=color,
                        label=f"{names[i]} accuracy")
        axes[1, 1].set_ylabel("Accuracy")
        axes[1, 1].set_xlabel("Epochs")
        axes[1, 1].legend(loc="best")

def plot_genotype(path, eval_every, print_genotype=True):
    with (open(path, 'rb')) as f:
        res = pickle.load(f)
        
    cwd = os.getcwd()
    if print_genotype:
        print(res['genotype'])
    
    for i in range(len(res['genotype'])):
        if i % eval_every == 0:
            
            display(Markdown(f'# Iteration: {i}'))
            if res['genotype'][i].normal is not None:
                plot(res['genotype'][i].normal, 'normal', 'normal cell')
                display(Image('normal.png'))
                os.remove(os.path.join(cwd, "normal"))
                os.remove(os.path.join(cwd, "normal.png"))
            
            if res['genotype'][i].reduce is not None:
                plot(res['genotype'][i].reduce, 'reduce', 'reduce cell')
                display(Image('reduce.png'))
                os.remove(os.path.join(cwd, "reduce"))
                os.remove(os.path.join(cwd, "reduce.png"))